In [ ]:
# 2025-12-12 against production
from csae_pyutils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist

from apis_core.apis_metainfo.models import Collection, Uri
from apis_core.apis_entities.models import Work, Person
from apis_core.apis_vocabularies.models import CollectionType, WorkType, PersonWorkRelation
from apis_core.apis_relations.models import PersonWork

from tqdm import tqdm

In [ ]:
df = gsheet_to_df("1-oRC9J8VjHvt2jym3AdZBWS3klZv3T_uT8rkUZHkb6I")

In [ ]:
df.head()

In [ ]:
col, _ = Collection.objects.get_or_create(name="Briefedition Wedekind")
domain = "wedekind"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = 'Frank Wedekinds Korrespondenz digital. <a href="https://briefedition.wedekind.fernuni-hagen.de">https://briefedition.wedekind.fernuni-hagen.de<a>'
col.collection_type = col_type
col.save()

In [ ]:
# process existing works first
data = {}
for i, row in tqdm(df.iterrows()):
    try:
        pmb_id = int(row["pmb_work"])
    except ValueError:
        continue
    try:
        entity = Work.objects.get(id=pmb_id)
        uri = row["uri"]
        pmb_uri, _ = Uri.objects.get_or_create(uri=uri, domain=domain, entity=entity)
        entity.collection.add(col)
    except ObjectDoesNotExist:
        print(pmb_id)
        continue
    

In [ ]:
# process new works now
data = {}
for i, row in tqdm(df.iterrows()):
    try:
        pmb_id = int(row["pmb_work"])
        continue
    except ValueError:
        domain_uri = row["uri"]
        try:
            year = f"{int(row["publication_year"])}"
        except ValueError:
            year = ""
        kind = row["category"]
        try:
            work_kind = WorkType.objects.get(name=kind)
        except ObjectDoesNotExist:
            kind = False
        try:
            uri = Uri.objects.get(uri=domain_uri)
            entity = uri.entity
            entity = Work.objects.get(id=entity.id)
        except ObjectDoesNotExist:
            if work_kind:    
                item = {
                    "name": row["title"],
                    "start_date_written": year,
                    "kind": work_kind
                }
            else:
                item = {
                    "name": row["title"],
                    "start_date_written": year
                }
            entity = Work.objects.create(**item)
            entity.collection.add(col)
            uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
            uri.entity = entity
            uri.save()

In [ ]:
person_work_rel_type, _ = PersonWorkRelation.objects.get_or_create(name="hat geschaffen")
person_work_rel_type

In [ ]:
# add authors
for i, row in tqdm(df.iterrows()):
    domain_uri = row["uri"]
    try:
        author_id = int(row["author_pmb_id"])
    except ValueError:
        continue
    try:
        int(row["pmb_work"])
        continue
    except ValueError:
        try:
            year = f"{int(row["publication_year"])}"
        except ValueError:
            year = ""
        uri = Uri.objects.get(uri=domain_uri)
        entity = uri.entity
        entity = Work.objects.get(id=entity.id)
        try:
            author = Person.objects.get(id=author_id)
        except ObjectDoesNotExist:
            continue
        item = PersonWork.objects.get_or_create(
            related_person=author,
            related_work=entity,
            relation_type=person_work_rel_type,
            start_date_written=year
        )        